# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил: Хайкин Глеб

### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
\min(\text{points}, 21)  \times 10 / 21,
$$

где points — количество баллов за домашнее задание, которое вы набрали. Максимальное число баллов, которое можно получить за решение данного домашнего задания — 24, все баллы сверх 21 идут в бонус (таким образом, за данное домашнее задание можно получить 3 бонусных балла). Накопленные бонусные баллы можно будет потом распределять по другим домашним заданиям и проверочным (+1 бонусный балл = +1 к оценке за домашнее задание/проверочную).

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format ='retina'

from tqdm import tqdm

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - `'title'`
 - `'description'`
 - `'Category_name'`
 - `'Category'`

Имеется информация об объектах 50 классов.
Задача: по новым объектам (`'title'`, `'description'`) предсказать `'Category'`.
(Очевидно, что параметр `'Category_name'` для предсказания классов использовать нельзя)

In [2]:
data = pd.read_csv("avito_data.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [3]:
data.shape

(30000, 4)

In [4]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

In [7]:
y_train[:5]

array([ 27,  20,  84, 106,  27])

### Токенизация (0.5 балла)


Токенизация — разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации. В данном задании мы будем использовать `WordPunctTokenizer`.

In [8]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text)
print("after:", tokenizer.tokenize(text.lower()))

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже.

In [9]:
def preprocess(text: str, tokenizer) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    
    return ' '.join(tokenizer.tokenize(text.lower()))

In [10]:
assert preprocess(text, tokenizer) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте `'title'` и `'description'` в `train` и `test`.

In [11]:
tokenizer = WordPunctTokenizer()
X_train = np.array([[preprocess(item[0], tokenizer), preprocess(item[1], tokenizer)] for item in X_train])
X_test = np.array([[preprocess(item[0], tokenizer), preprocess(item[1], tokenizer)] for item in X_test])

In [12]:
assert X_train[5][0] == '1 - к квартира , 33 м² , 4 / 5 эт .'
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'
assert X_test[2][0] == 'фара правая toyota rav 4 галоген 2015 - 19'
assert X_test[2][1] == 'фара правая для toyota rav4 2015 / оригинальный номер : 8113042650 / тойота рав4 тоета рав 4 / производитель : toyota / состояние : отличное без дефектов ! / комментарий : после 2015 не ксенон галоген + диод / пожалуйста , уточняйте соответствие вашего заказа изображенному на фото . / звоните уточняйте по наличию предоставляется время на проверку детали / отправляем в регионы рф транспортными компаниями / . / всегда включен вайбер вацап по вопросам !/ дополнительное фото по запросу'

### BOW (3 балла)

Один из традиционных подходов — построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в `train data`
 - Для каждого примера из `train` посчитать, сколько раз каждое слово из словаря в нём встречается


 В `sklearn` есть `CountVectorizer`, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где каждому токену соответствует количество раз, которое оно встретилось в `X_train`.

In [13]:
tokens_cnt = dict()

In [14]:
for title, description in tqdm(X_train):
    string = title + ' ' + description
    for word in string.split():
         tokens_cnt[word] = tokens_cnt.get(word, 0) + 1

100%|██████████| 21000/21000 [00:01<00:00, 14837.65it/s]


In [15]:
assert tokens_cnt['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов.

In [16]:
sorted_tokens = sorted(tokens_cnt, key=lambda x: tokens_cnt.get(x), reverse=True)

In [17]:
print(f'10 самых частых токенов: {sorted_tokens[:10]}')

10 самых частых токенов: ['/', ',', '.', '-', 'в', 'и', 'на', './', ':', 'с']


In [18]:
print(f'10 самых редких токенов: {sorted_tokens[::-1][:10]}')

10 самых редких токенов: ['фрионом', 'хлебозаводская', 'дооснастить', 'беспрецедентно', 'понравившейся', 'объективную', 'столиц', 'петровского', 'гремят', 'шуршат']


__Задание:__ оставьте в словаре только топ-10000 самых частотных токенов, также создайте отдельный список из этих слов.

1. Оставим только топ-10000 самых частотных токенов:

In [19]:
for token in sorted_tokens[10000:]:
    tokens_cnt.pop(token)

2. Cоздадим отдельный список из этих слов

In [20]:
tokens_list = sorted_tokens[:10000]
tokens_list

['/',
 ',',
 '.',
 '-',
 'в',
 'и',
 'на',
 './',
 ':',
 'с',
 '(',
 'по',
 'для',
 'не',
 ')',
 '2',
 '1',
 '!',
 'до',
 'от',
 '—',
 '"',
 '3',
 '5',
 'состоянии',
 '!/',
 'у',
 'за',
 'все',
 'размер',
 'без',
 'из',
 '*',
 '4',
 'есть',
 'доставка',
 '00',
 'см',
 'к',
 'продам',
 'состояние',
 'или',
 ')/',
 '10',
 'цена',
 ':/',
 'б',
 '•',
 'наличии',
 'руб',
 '6',
 'при',
 'очень',
 'р',
 'фото',
 'новые',
 'звоните',
 'можно',
 'м',
 'запчасти',
 ';/',
 '+',
 'новый',
 '8',
 'отличном',
 '20',
 '7',
 'продаю',
 'хорошем',
 'гарантия',
 'как',
 'под',
 '9',
 ';',
 'а',
 '),',
 'так',
 '–',
 'мы',
 'г',
 '«',
 'платье',
 'мм',
 'работы',
 'оригинал',
 ').',
 'квартира',
 'россии',
 'вы',
 'торг',
 'цвет',
 'комплект',
 'м²',
 'новая',
 '50',
 '↓',
 '100',
 'только',
 'более',
 'номер',
 'шт',
 '%',
 'запчастей',
 'регионы',
 'лет',
 '0',
 'дом',
 'года',
 '!!!',
 'что',
 'также',
 'вас',
 'отличное',
 'ул',
 '15',
 'любой',
 'работаем',
 '30',
 'длина',
 'куртка',
 '12',
 'рубле

__Задание:__ реализуйте функцию, которая переводит текст в вектор из чисел. То есть каждому токену из списка токенов сопоставляется количество раз, которое он встретился в тексте.

In [22]:
def text_to_bow(text: str, tokens_list: list) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указано количество его употреблений в предложении
    input: строка, список токенов
    output: вектор той же размерности, что и список токенов
    """
    string = text.split()
    # создаем вектор той же размерности, что и список токенов
    vec = np.zeros(len(tokens_list))
    for word in string:
        if word in tokens_list:
            # учитываем появление слова в векторе 
            vec[tokens_list.index(word)] += 1
    
    return vec

In [23]:
example_text = text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml", tokens_list)

assert np.allclose(example_text.mean(), 0.0008)

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и каждому тексту из `'description'` сопоставляет вектор.

In [24]:
def descr_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого описания товара возвращает вектор его bow """
    return np.array([text_to_bow(description, tokens_list) for title, description in tqdm(items)])

In [25]:
X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

100%|██████████| 9000/9000 [00:33<00:00, 265.05it/s]


In [26]:
assert X_train_bow.shape == (21000, 10000), X_test_bow.shape == (9000, 10000)
assert 0.005 < X_train_bow.mean() < 0.006
assert 0.005 < X_test_bow.mean() < 0.006

### Логистическая регрессия и SVM (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве — какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVM с линейным ядром (`sklearn.svm.LinearSVC` или `sklearn.svm.SVC(kernel='linear')`) с базовыми параметрами. При необходимости можете увеличить максимальное число итераций. В качестве `random_state` возьмите 13.

_Подсказка: для того, чтобы было проще обучать, можно использовать [разреженные матрицы](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D1%80%D0%B5%D0%B6%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0), многие модели из `sklearn` умеют с ними работать. Соответствующий модуль из `scipy`: [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html). Нетрудно заметить, что в полученных BOW-матрицах очень много нулей. Если хранить в памяти только ненулевые элементы, можно сильно оптимизировать вычисления. Можете в этом убедиться:_

In [27]:
print('Train array in memory (raw): {:.3f} Mb'.format(X_train_bow.nbytes * 1e-6))

from scipy.sparse import csr_matrix
X_train_bow_csr = csr_matrix(X_train_bow)
print('Train array in memory (compressed): {:.3f} Mb'.format(
    (X_train_bow_csr.data.nbytes + X_train_bow_csr.indptr.nbytes + X_train_bow_csr.indices.nbytes) * 1e-6)
)

Train array in memory (raw): 1680.000 Mb
Train array in memory (compressed): 8.606 Mb


In [28]:
X_test_bow_csr = csr_matrix(X_test_bow)

In [29]:
from sklearn.metrics import accuracy_score

<br>1. Логистическая регрессия 

In [30]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=13)
lr.fit(X_train_bow_csr, y_train)

/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
y_pred = lr.predict(X_test_bow_csr)

assert accuracy_score(y_test, y_pred) > 0.695

In [32]:
accuracy_1_logreg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_1_logreg}")

Accuracy: 0.7011111111111111


<br>2. SVM

In [33]:
from sklearn.svm import LinearSVC
SVM_linear_classifier = LinearSVC(random_state=13)
SVM_linear_classifier.fit(X_train_bow_csr, y_train)

/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=13, tol=0.0001,
          verbose=0)

In [34]:
y_pred = SVM_linear_classifier.predict(X_test_bow_csr)

assert accuracy_score(y_test, y_pred) > 0.68

In [35]:
accuracy_1_svm = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_1_svm}")

Accuracy: 0.684


### Модификация признаков (2 балла)

<br>1. Прибавьте к соответствующим BOW-векторам BOW-вектора для `'title'` товара с некоторым весом. Изменится ли качество? Как вы можете это объяснить?

Для того чтобы создать BOW-вектора для `'title'` необходимо определить функцию `title_to_bow` по аналогии с `descr_to_bow`. Сделаем это.

In [36]:
def title_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого заглавия товара возвращает вектор его bow """
    return np.array([text_to_bow(title, tokens_list) for title, description in tqdm(items)])

In [37]:
X_train_title_bow = title_to_bow(X_train, tokens_list)
X_test_title_bow = title_to_bow(X_test, tokens_list)

100%|██████████| 9000/9000 [00:08<00:00, 1015.46it/s]


Теперь прибавим к соответствующим BOW-векторам BOW-вектора для 'title' товара с весом большем единицы. Например, с весом 2 (будем считать заголовок в 2 раза релевантнее описания).

In [38]:
X_train_bow_full = 2 * X_train_title_bow + X_train_bow
X_test_bow_full = 2 * X_test_title_bow + X_test_bow

Будем также использовать разреженные матрицы.

In [39]:
X_train_bow_csr = csr_matrix(X_train_bow_full)
X_test_bow_csr = csr_matrix(X_test_bow_full)

Проверим, увеличилось ли качество.

a) Логистическая регрессия 

In [40]:
lr = LogisticRegression(random_state=13, max_iter=500)
lr.fit(X_train_bow_csr, y_train)
y_pred = lr.predict(X_test_bow_csr)
accuracy_2_logreg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_2_logreg}")

Accuracy: 0.791


/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [41]:
print(f"Относительное улучшение: {round((accuracy_2_logreg - accuracy_1_logreg) / accuracy_1_logreg, 2)}%")

Относительное улучшение: 0.13%


b) SVM

In [42]:
SVM_linear_classifier = LinearSVC(random_state=13)
SVM_linear_classifier.fit(X_train_bow_csr, y_train)
y_pred = SVM_linear_classifier.predict(X_test_bow_csr)
accuracy_2_svm = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_2_svm}")

Accuracy: 0.7605555555555555


/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [43]:
print(f"Относительное улучшение: {round((accuracy_2_svm - accuracy_1_svm) / accuracy_1_svm, 2)}%")

Относительное улучшение: 0.11%


Видно, что качество у обоих методов выросло. Обычно `'title'` — это обобщение `'description'`. Нередко в `'title'` пишется как раз товар, который мы пытаем категоризовать, а в `'description'` — какие-то его характеристики. Следовательно, наша обучающая выборка после прибавления к BOW-векторам для `'description'` BOW-вектора для 'title' становится более релевантной, информативной.

<br>2. Нормализуйте данные с помощью `MinMaxScaler` или `MaxAbsScaler` перед обучением. Что станет с качеством и почему?

In [45]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

In [46]:
X_train_scaled = scaler.fit_transform(X_train_bow_full)
X_test_scaled = scaler.transform(X_test_bow_full)
X_train_scaled_csr = csr_matrix(X_train_scaled)
X_test_scaled_csr = csr_matrix(X_test_scaled)

Проверим, увеличилось ли качество.

a) Логистическая регрессия

In [47]:
lr = LogisticRegression(random_state=13, max_iter=500)
lr.fit(X_train_scaled_csr, y_train)
y_pred = lr.predict(X_test_scaled_csr)
accuracy_3_logreg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_3_logreg}")

Accuracy: 0.7477777777777778


In [48]:
print(f"Относительное улучшение при сравнении с моделью без модификаций: {round((accuracy_3_logreg - accuracy_1_logreg) / accuracy_1_logreg, 2)}%")

Относительное улучшение при сравнении с моделью без модификаций: 0.07%


Для логистической регресси при нормализации данных качество упало, если сравнивать с первым пунктом, но зато теперь решение сходиться к локальному минимуму.

In [49]:
print(f"Относительное ухудшение при сравнении с моделью с первой модификацией: {round((accuracy_3_logreg - accuracy_2_logreg) / accuracy_2_logreg, 2)}%")

Относительное ухудшение при сравнении с моделью с первой модификацией: -0.05%


Возможно ухудшение связано с тем, что нормальные признаки имели достаточно большой размах, а шумовые признаки — маленький. При масштабировании же мы привели все признаки к единому масштабу и от этого ухудшилось качество в логистической регрессии.

б) SVM

In [50]:
SVM_linear_classifier = LinearSVC(random_state=13)
SVM_linear_classifier.fit(X_train_scaled_csr, y_train)
y_pred = SVM_linear_classifier.predict(X_test_scaled_csr)
accuracy_3_svm = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_3_svm}")

Accuracy: 0.7767777777777778


In [51]:
print(f"Относительное улучшение при сравнении с моделью без модификаций: {round((accuracy_3_svm - accuracy_1_svm) / accuracy_1_svm, 2)}%")

Относительное улучшение при сравнении с моделью без модификаций: 0.14%


In [52]:
print(f"Относительное улучшение при сравнении с моделью с первой модификацией: {round((accuracy_3_svm - accuracy_2_svm) / accuracy_2_svm, 2)}%")

Относительное улучшение при сравнении с моделью с первой модификацией: 0.02%


Для SVM при нормализации данных качество улучшилось и теперь решение сходиться к минимуму.

Почему в данном случае использовать `StandardScaler` — не очень хорошая идея?

Возможно, из-за деления на ноль (когда дисперсия равна нулю): в таком случае будет вылезать nan.

### Иная предобработка (1 балл)

**На выбор**:

- **либо** обучите модели, используя для предобработки токенизатор и лемматизатор `pymystem3.Mystem`.
- **либо** добавьте к предобработке стэмминг.

Сравните полученное сейчас качество с полученным ранее и сделайте вывод.

Выберем в качестве дополнительной предоработки лемматизатор.

In [53]:
from pymystem3 import Mystem
mystem_analyzer = Mystem(entire_input=False)

Будем лемматизировать поочередно все объявления (`string`) в обучающей выборке и добавлять в словарь `tokens_cnt` какждое новое вхождение конкретного слова.

In [54]:
tokens_cnt_lemmatized = {}
for title, description in tqdm(X_train):
    string = title + ' ' + description
    for word in mystem_analyzer.lemmatize(string):
         tokens_cnt_lemmatized[word] = tokens_cnt_lemmatized.get(word, 0) + 1

100%|██████████| 21000/21000 [00:44<00:00, 477.03it/s]


Отсортируем наш словарь и оставим только топ-10000 слов, как делали это и раньше.

In [55]:
sorted_tokens_lemmatized = sorted(tokens_cnt_lemmatized, key=lambda x: tokens_cnt_lemmatized.get(x), reverse=True)

In [56]:
tokens_list_lemmatized = sorted_tokens_lemmatized[:10000]
tokens_list_lemmatized

['в',
 'и',
 'на',
 'с',
 'по',
 'для',
 'состояние',
 'новый',
 'не',
 'продавать',
 'весь',
 'размер',
 'до',
 'от',
 'быть',
 'цена',
 'у',
 'доставка',
 'за',
 'без',
 'вы',
 'хороший',
 'отличный',
 'из',
 'год',
 'наш',
 'запчасть',
 'наличие',
 'см',
 'магазин',
 'к',
 'мы',
 'или',
 'любой',
 '═',
 'автомобиль',
 'б',
 'квартира',
 'звонить',
 'руб',
 'при',
 'комплект',
 'очень',
 'компания',
 'гарантия',
 'работа',
 'дом',
 'р',
 'фото',
 'товар',
 'можно',
 'м',
 'мочь',
 'работать',
 'цвет',
 'ваш',
 'другой',
 'один',
 'платье',
 'телефон',
 'задний',
 'регион',
 'торг',
 'возможный',
 'под',
 'день',
 'как',
 'номер',
 'большой',
 'все',
 'россия',
 'транспортный',
 'а',
 'детский',
 'так',
 'оригинал',
 'г',
 'мм',
 'подходить',
 '↓',
 'передний',
 'город',
 'система',
 'шина',
 'раз',
 'двигатель',
 'зимний',
 'скидка',
 'продажа',
 'ремонт',
 'два',
 'модель',
 'москва',
 'диск',
 '✔',
 'полный',
 'деталь',
 'куртка',
 'только',
 'много',
 'удобный',
 'адрес',
 'оригин

Переопределим функции `text_to_bow`, `descr_to_bow` и `title_to_bow` согласно лемматизации.

In [57]:
def text_to_bow(text: str, tokens_list_lemmatized: list) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указано количество его употреблений в предложении
    input: строка, список токенов
    output: вектор той же размерности, что и список токенов
    """
    # создаем вектор той же размерности, что и список токенов
    vec = np.zeros(len(tokens_list_lemmatized))
    for word in text:
        if word in tokens_list_lemmatized:
            # учитываем появление слова в векторе 
            vec[tokens_list_lemmatized.index(word)] += 1
    
    return vec

def descr_to_bow(items: np.array, tokens_list_lemmatized: list) -> np.array:
    """ Для каждого описания товара возвращает вектор его bow """
    return np.array([text_to_bow(mystem_analyzer.lemmatize(description),
                                 tokens_list_lemmatized) for title, description in tqdm(items)])

def title_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого заглавия товара возвращает вектор его bow """
    return np.array([text_to_bow(mystem_analyzer.lemmatize(title),
                                 tokens_list_lemmatized) for title, description in tqdm(items)])

In [58]:
X_train_descr = descr_to_bow(X_train, tokens_list_lemmatized)
X_test_descr = descr_to_bow(X_test, tokens_list_lemmatized)
X_train_title = title_to_bow(X_train, tokens_list_lemmatized)
X_test_title = title_to_bow(X_test, tokens_list_lemmatized)

100%|██████████| 9000/9000 [00:04<00:00, 2011.03it/s]


Сложим два BOW-вектора, чтобы в итоге получить BOW-вектор одного объявления. Так же — как и раньше, — будем считать, что заголовок объявления в 2 раза важнее самого описания товара.

In [59]:
X_train_bow = X_train_descr + 2 * X_train_title
X_test_bow = X_test_descr + 2 * X_test_title

Также воспользуемся разреженными матрицами.

In [60]:
X_train_bow_csr = csr_matrix(X_train_bow)
X_test_bow_csr = csr_matrix(X_test_bow)

Теперь проверем качество моделей.

<br> a) Логистическая регрессия

In [80]:
lr = LogisticRegression(C=0.3, random_state=13)
lr.fit(X_train_bow_csr, y_train)
y_pred = lr.predict(X_test_bow_csr)
accuracy_4_logreg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_4_logreg}")

Accuracy: 0.8054444444444444


/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



Качество выросло. Теперь посмотрим на относительное увеличение качества при сравнении с первым вариантом алгоритма.

In [81]:
print(f"Относительное улучшение при сравнении с моделью без модификаций: {round((accuracy_4_logreg - accuracy_1_logreg) / accuracy_1_logreg, 2)}%")

Относительное улучшение при сравнении с моделью без модификаций: 0.15%


<br> б) SVM

Нормализуем данные для SVM

In [69]:
scaler = MaxAbsScaler()
X_train_scaled_csr = scaler.fit_transform(X_train_bow_csr)
X_test_scaled_csr = scaler.transform(X_test_bow_csr)

In [70]:
SVM_linear_classifier = LinearSVC(C=0.3, random_state=13)
SVM_linear_classifier.fit(X_train_scaled_csr, y_train)
y_pred = SVM_linear_classifier.predict(X_test_scaled_csr)
accuracy_4_svm = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_4_svm}")

Accuracy: 0.795


Качество выросло. Теперь посмотрим на относительное увеличение качества при сравнении с первым вариантом алгоритма.

In [71]:
print(f"Относительное улучшение при сравнении с моделью без модификаций: {round((accuracy_4_svm - accuracy_1_svm) / accuracy_1_svm, 2)}%")

Относительное улучшение при сравнении с моделью без модификаций: 0.16%


**Вывод**: используя лемматизацию, наши модели улучшились, что и логично. Поскольку лемматизация — это процесс приведения разных форм одного слова к начальной форме, то есть лемме. В объявлении одно и то же слово может употребляться в разных наклонениях, падежах и т.д. — есть риск недосчитать важность какого-то релевантного слова.

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В `sklearn` есть `TfidfVectorizer`, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для `'title'` и `'description'` (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация `'title'` и `'description'`).

__Задание:__ составьте словарь, где каждому слову из изначального списка будет соответствовать количество документов из `train`-части, где это слово встретилось.

In [72]:
word_document_cnt = {}
for title, description in tqdm(X_train):
    document = title + ' ' + description
    # set() нужен для того, чтобы не учитывать больше одного раза слово,
    # присутсвующее несколько раз в одном документе 
    for word in set(document.split()):
         word_document_cnt[word] = word_document_cnt.get(word, 0) + 1

100%|██████████| 21000/21000 [00:01<00:00, 12065.21it/s]


In [73]:
assert word_document_cnt['размер'] == 2839

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. Для вычисления IDF также необходимо число документов в `train`-части (параметр `n_documents_total`).

In [74]:
def text_to_tfidf(text: str, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    text = text.split()
    vec = np.zeros(len(tokens_list))
    
    # пробегаем по каждому слову в строке
    for term in set(text):
        # если это слово есть в нашем топ-10000 списке, считаем для него TF-IDF
        if term in tokens_list:
            TF_t = text.count(term) / len(set(text))
            IDF_t = np.log(n_documents_total / word_document_cnt[term])
            # присваеваем индексу вектора — конкретному слову в этом векторе — значение TF-IDF
            vec[tokens_list.index(term)] = TF_t * IDF_t
        
    return vec

In [75]:
example_text = text_to_tfidf(
    'сдаётся уютный , тёплый гараж для стартапов в ml',
    word_document_cnt,
    tokens_list,
    n_documents_total=len(X_train)
)
assert 0.0003 < example_text.mean() < 0.0004

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf. В качестве текстов используйте конкатенацию `'title'` и `'description'`.

In [76]:
def items_to_tfidf(items: np.array, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Для каждого товара возвращает его tf-idf вектор
    """
      
    return np.array([text_to_tfidf(title + " " + description,
                                   word_document_cnt,
                                   tokens_list,
                                   n_documents_total) for title, description in tqdm(items)])

In [77]:
X_train_tfidf = items_to_tfidf(X_train, word_document_cnt, tokens_list, len(X_train))
X_test_tfidf = items_to_tfidf(X_test, word_document_cnt, tokens_list, len(X_train))

100%|██████████| 9000/9000 [00:37<00:00, 238.52it/s]


In [78]:
assert X_train_tfidf.shape == (21000, 10000), X_test_tfidf.shape == (9000, 10000)
assert 0.0002 < X_train_tfidf.mean() < 0.0004
assert 0.0002 < X_test_tfidf.mean() < 0.0004

Воспользуемся разреженными матрицами.

In [79]:
X_train_tfidf_csr = csr_matrix(X_train_tfidf)
X_test_tfidf_csr = csr_matrix(X_test_tfidf)

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score). Сделайте вывод.

<br> a) Логистическая регрессия

In [84]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_tfidf_csr, y_train)
y_pred = lr.predict(X_test_tfidf_csr)
accuracy_5_logreg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_5_logreg}")

Accuracy: 0.7311111111111112


/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [85]:
assert accuracy_score(y_test, y_pred) > 0.675

In [86]:
print(f"Относительное улучшение при сравнении с первой моделью: {round((accuracy_5_logreg - accuracy_1_logreg) / accuracy_1_logreg, 2)}%")
print(f"Относительное ухудшение при сравнении с четвертой моделью (bow-вектора и лемматизация): {round((accuracy_5_logreg - accuracy_4_logreg) / accuracy_4_logreg, 2)}%")

Относительное улучшение при сравнении с первой моделью: 0.04%
Относительное ухудшение при сравнении с четвертой моделью (bow-вектора и лемматизация): -0.09%


<br> б) SVM

In [87]:
SVM_linear_classifier = LinearSVC(random_state=13, max_iter=200)
SVM_linear_classifier.fit(X_train_scaled_csr, y_train)
y_pred = SVM_linear_classifier.predict(X_test_scaled_csr)
accuracy_5_svm = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_5_svm}")

Accuracy: 0.7923333333333333


In [88]:
assert accuracy_score(y_test, y_pred) > 0.79

In [89]:
print(f"Относительное улучшение при сравнении с первой моделью: {round((accuracy_5_svm - accuracy_1_svm) / accuracy_1_svm, 2)}%")
print(f"Относительное ухудшение при сравнении с четвертой моделью (bow-вектора и лемматизация): {round((accuracy_5_svm - accuracy_4_svm) / accuracy_4_svm, 5)}%")

Относительное улучшение при сравнении с первой моделью: 0.16%
Относительное ухудшение при сравнении с четвертой моделью (bow-вектора и лемматизация): -0.00335%


**Вывод**: bow-вектора в сочетании с лемматизацией лучше TF-IDF.

### Word Vectors (4 балла)

Давайте попробуем другой подход — каждому слову сопоставим какое-то векторное представление (эмбеддинг) — но достаточно маленькой размерности. Таким образом мы сильно уменьшим количество параметров в модели.

Почитать про это подробнее можно тут:

- https://habr.com/ru/company/ods/blog/329410/

Вектора мы возьмём уже готовые (обученные на текстах из интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [95]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-14 19:46:14--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-14 19:46:15--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9c5161deffa507fea87f11afd8.dl.dropboxusercontent.com/cd/0/inline/A10A5d72axWk8fNGTC-8opH-bFupom45H37XWfyerZLZLiFxnXE-FTmViuHMUVOtlKIUlBqcoxvjmX2bVvSLuU0qKpDJOeEaKoE3GZH3Y1Vgtw/file# [following]
--2020-04-14 19:46:15--  https://uc9c5161deffa507fea87f11afd8.dl.dropboxusercontent.com/cd/0/inline/A10A5d72axWk8fNGTC-8opH-bFupom45H37XWfyerZLZLiFxnXE-FTmViuHMUVOtlKIUlBqcoxvjmX2bVvSLuU0qKpDJOeEaKoE3GZH3Y1Vgtw/file
Resolving uc9c5161deffa507fea87f11afd8.dl.dropboxuser

In [102]:
!tar -xzf ru.tar.gz.1

In [88]:
!pip install gensim

     |████████████████████████████████| 23.7 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 105 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 2.9 MB/s eta 0:00:01     |█████████████████████▏          | 4.0 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 2.6 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.11.1-py3-none-any.whl size=95255 sha256=c6e6a57281bf6319974e29683d08b3589212c26aad23865c80a2769cb15feb45
  Stored in directory: /Users/khaykingleb/Library/Caches/pip/wheels/1a/8c/a2/7b24df77c58dab0aec275c1bd3d4392d54df941020199f6759
Successfully built smart-open
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


In [90]:
import gensim
from gensim.models.wrappers import FastText

embedding_model = FastText.load_fasttext_format('ru.bin')

In [91]:
# как мы видим, каждому слову данная модель сопоставляет вектор размерности 300
print(embedding_model['привет'].shape)
print(embedding_model['привет'])

(300,)
[ 0.02916384  0.02167605  0.05127367 -0.00971958  0.0465235  -0.03945766
  0.02737866  0.00638128 -0.03774629 -0.04257201 -0.00995653  0.02291315
 -0.02301722  0.06697998 -0.03674482 -0.02403202 -0.05404469  0.01372932
  0.00926399 -0.0013149   0.11941359 -0.022448    0.04011497  0.06980549
  0.00407011 -0.09384539  0.03050164 -0.02578281 -0.03525181 -0.06603175
  0.04752798  0.05874675  0.01983666  0.06092105 -0.00957561  0.08307806
 -0.01288903  0.04705157  0.02198839 -0.00649013 -0.0171444   0.03302203
  0.02124882 -0.01902875 -0.05235172  0.03458685 -0.01409259 -0.07477519
  0.01916078  0.02985001  0.0086322   0.03051201  0.02831862  0.04549561
  0.00761138 -0.05459622  0.09056009 -0.08807947 -0.05420396 -0.04793203
 -0.05672329 -0.03025264 -0.03024072 -0.05890108 -0.03137474  0.03292617
  0.05440779 -0.04548327 -0.07266086 -0.09327219  0.07247883  0.0111061
  0.01824225 -0.10570452  0.05110046 -0.04659343 -0.03277056 -0.00803401
 -0.03978698  0.00826598 -0.01074128  0.01843

__Задание:__ реализуйте функцию, выдающую эмбеддинг для предложения — как сумму эмбеддингов токенов.

In [95]:
import re

In [96]:
def sentence_embedding(sentence: str, embedding_model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    sent_embed = np.zeros(300)
    # поскольку для знаков пунктуации и нерусских слов нету векторного представления в embedding_model,
    # нам необходимо "почистить" наш текст посредством регулярных выражений
    prog = re.compile('[А-Яа-яё\-]+')
    
    for word in prog.findall(sentence):
        try:
            sent_embed += embedding_model[word]
        except KeyError:
            pass # Исключаем иные слова (наподобие зп), которые также не имеют векторного представления
    
    return sent_embed

In [97]:
assert sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model).shape == (300,)
assert np.allclose(np.linalg.norm(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model)), 2.6764746)

__Задание:__ сделайте все то же, что в предыдущих пунктах — реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество. Сделайте вывод, что работает лучше: модель, основанная на TF-IDF, или модель, обученная на предобученных эмбеддингах?

<br>1. Реализуем функцию, которая преобразует данные.

In [98]:
def items_to_embedding(items: np.array, embedding_model) -> np.array:
    return np.array([sentence_embedding(title + " " + description,
                                        embedding_model) for title, description in tqdm(items)])

In [99]:
X_train_embed = items_to_embedding(X_train, embedding_model)
X_test_embed = items_to_embedding(X_test, embedding_model)

100%|██████████| 9000/9000 [00:09<00:00, 999.27it/s] 


In [100]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(csr_matrix(X_train_embed))
X_test_scaled = scaler.transform(csr_matrix(X_test_embed))

<br> a) Логистическая регрессия

In [101]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_scaled, y_train)
y_pred = lr.predict(X_test_scaled)
accuracy_6_logreg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_6_logreg}")

Accuracy: 0.4201111111111111


/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [102]:
print(f"Относительное ухудшение при сравнении с пятой моделью (TF-IDF): {round((accuracy_6_logreg - accuracy_5_logreg) / accuracy_5_logreg, 2)}%")

Относительное ухудшение при сравнении с пятой моделью (TF-IDF): -0.43%


<br> б) SVM

In [103]:
SVM_linear_classifier = LinearSVC(random_state=13, max_iter=200)
SVM_linear_classifier.fit(X_train_scaled, y_train)
y_pred = SVM_linear_classifier.predict(X_test_scaled)
accuracy_6_svm = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_6_svm}")

/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.5164444444444445


In [104]:
print(f"Относительное ухудшение при сравнении с пятой моделью (TF-IDF): {round((accuracy_6_svm - accuracy_5_svm) / accuracy_5_svm, 2)}%")

Относительное ухудшение при сравнении с пятой моделью (TF-IDF): -0.35%


**Вывод**: При сравнении с моделью, основанной на TF-IDF, модель, обученная на предобученных эмбеддингах, явно уступает.

### Что дальше? (8 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать n-gram модели текстовой классификации (__2 балла__)

2. Поработать с другими эмбеддингами для слов (например `word2vec` или `GloVe`) (__2 балла__)

3. Применить другие способы токенизации (например, `pymorphy2`, `spaCy`) и в целом предобработки данных (стоп-слова, стэмминг, лемматизация) (__2 балла__)

4. Добиться качества > 0.81 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1 балл__)

4. Добиться качества > 0.82 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1 балл__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

<br>I. Удалим ненужные слова — воспользуемся стоп-словами — и пунктуацию в объявлениях (__третий пункт__) и снова лемматизируем наш текст. Также реализуем n-gram модель текстовой классификации (__первый пункт__).

In [36]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import TweetTokenizer

In [37]:
from pymystem3 import Mystem
mystem_analyzer = Mystem(entire_input=False)
noise = stopwords.words('russian') + list(punctuation)

In [38]:
def del_noise(text: list, noise: list):
    return " ".join([token.lower() for token in text if token not in noise and len(token) > 2])

def preprocess(X, noise: list):
    return np.array([del_noise(mystem_analyzer.lemmatize(title), noise) 
                     + " " +
                     del_noise(mystem_analyzer.lemmatize(description), noise) for title, description in tqdm(X)])

In [39]:
X_train_preproc = preprocess(X_train, noise)
X_test_preproc = preprocess(X_test, noise)

100%|██████████| 9000/9000 [00:30<00:00, 291.59it/s]


Создадим словарь и оставим только топ-20000 самых частых слов, а также создам отдельный список из этих слов.

In [40]:
tokens_cnt = dict()
for text in X_train_preproc:
    for term in text.split():
         tokens_cnt[term] = tokens_cnt.get(term, 0) + 1

In [41]:
sorted_tokens = sorted(tokens_cnt, key=lambda x: tokens_cnt.get(x), reverse=True)
tokens_list = sorted_tokens[:20000]

Реализуем n-gram модель текстовой классификации.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
# реализуем n-граммы
tfidf_vec_ngram = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), vocabulary=tokens_list)
X_train_vec = tfidf_vec_ngram.fit_transform(X_train_preproc)
X_test_vec = tfidf_vec_ngram.transform(X_test_preproc)

a) Логистическая регрессия

In [44]:
lr = LogisticRegression(random_state=13)
lr.fit(csr_matrix(X_train_vec), y_train)
y_pred = lr.predict(csr_matrix(X_test_vec))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.7828888888888889


/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


b) SVM:

In [47]:
scaler = MaxAbsScaler()
X_train_vec_scaled = csr_matrix(scaler.fit_transform(X_train_vec))
X_test_vec_scaled = csr_matrix(scaler.transform(X_test_vec))

In [48]:
SVM_linear_classifier = LinearSVC(C=0.145, random_state=13)
SVM_linear_classifier.fit(X_train_vec_scaled, y_train)
y_pred = SVM_linear_classifier.predict(X_test_vec_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.8203333333333334


__Вывод__: комбинация стоп-слов, лемматизации, выброса пунктуации и слов длинной в 2 символа с TF-IDF векторами в сочетании дала неплохое качество. Теперь попробуем его дальше улучшить.

Так же мы достигли качества в 0.82 и, тем самым, выполнили __четырый__ и __пятый__ пункты.

<br>II. Поработаем с эмбеддингом `Word2Vec` (__второй пункт__). 

In [49]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

Представим `X_train_lemmatized` и `X_test_lemmatized` как лист листов.

In [50]:
X_train_new = [text.split() for text in X_train_preproc]
X_test_new = [text.split() for text in X_test_preproc]

In [52]:
model = Word2Vec(X_train_new, size=200, min_count=1, workers=cpu_count())

In [53]:
 model.most_similar('iphone')

/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('apple', 0.9374587535858154),
 ('gray', 0.9266039729118347),
 ('samsung', 0.9129846096038818),
 ('galaxy', 0.9117615818977356),
 ('ростест', 0.9086523056030273),
 ('silicone', 0.9059789180755615),
 ('plus', 0.9035242199897766),
 ('xiaomi', 0.8967676162719727),
 ('отвязывать', 0.8921439051628113),
 ('min', 0.888987123966217)]

In [54]:
word2vec_ndim = model['iphone'].shape

/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


Мы обучили модель `Word2Vec` для нашей обучающей выборки. Теперь реализуем ее.

In [55]:
def sentence_embedding(sentence: str, embedding_model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    sent_embed = np.zeros(word2vec_ndim)
    
    for word in sentence.split():
        try:
            sent_embed += embedding_model[word]
        except KeyError:
            pass
    
    return sent_embed

def items_to_embedding(items: np.array, embedding_model) -> np.array:
    return np.array([sentence_embedding(title + " " + description,
                                        embedding_model) for title, description in tqdm(items)])

In [56]:
X_train_embed = items_to_embedding(X_train, model)
X_test_embed = items_to_embedding(X_test, model)

  0%|          | 0/21000 [00:00<?, ?it/s]/opt/anaconda3/envs/r_env/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
100%|██████████| 9000/9000 [00:02<00:00, 3064.77it/s]


In [57]:
X_train_crs = csr_matrix(scaler.fit_transform(X_train_embed))
X_test_crs = csr_matrix(scaler.transform(X_test_embed))

In [58]:
SVM_linear_classifier = LinearSVC(C=0.145, random_state=13)
SVM_linear_classifier.fit(X_train_crs, y_train)
y_pred = SVM_linear_classifier.predict(X_test_crs)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.5157777777777778


__Вывод__: Даже обученный на наших данных, а не готовый, эмбеддинг не показал хороших результатов. Возможно, это следствие того, что корпус текстов относительно мал.